In [ ]:
import torch
from torch import nn
import torch.nn.functional as func
device = 'cuda' if torch.cuda.is_available() else 'cpu'
block_size = 8
batch_size = 4

In [ ]:
with open('LLM_dataset.txt', 'r', encoding='utf-8') as f:
  text = f.read()

chars = sorted(set(text))
vocabulary_size = len(chars)
len(text)

1083285

In [ ]:
string_to_int = { ch:i for i,ch in enumerate(chars) }
int_to_string = { i:ch for i,ch in enumerate(chars) }
encode = lambda s: [string_to_int[c] for c in s]
decode = lambda l: ''.join([int_to_string[i] for i in l])

data = torch.tensor(encode(text), dtype=torch.long)
# print(decode([38, 54, 71, 72]))
# print(int_to_string[38])
# print(decode([54, 73, 11,  1, 27, 58, 60,  1]))

In [ ]:
n = int(0.8*len(data))
train_data = data[:n]
val_data = data[n:]

torch.manual_seed(23)
def get_batch(split): #just gets random chunks of numbers from the dataset
  data = train_data if split == 'train' else val_data # what data to use train or validate
  ix = torch.randint(len(data)-block_size, (batch_size,))
  print(ix)
  x = torch.stack([data[i:i+block_size] for i in ix])
  y = torch.stack([data[i+1:i+block_size+1] for i in ix])
  x, y = x.to(device), y.to(device)
  return x, y

x, y = get_batch('train')

# x.dtype
print(decode(y[0].tolist()))
# print(decode(x))

tensor([630431,  38326, 125756, 695577])
ut
4 fee


In [ ]:
@torch.no_grad()
def estimate_loss():
  out = {}
  model.eval()
  for split in ['train', 'val']:
    losses = torch.zeros(250)
    for k in range(250):
      X, Y = get_batch(split)
      logits, loss = model(X, Y)
      losses[k] = loss.item()
    out[split] = loss.mean()
  model.train()
  return out

In [ ]:
torch.manual_seed(23)

class LanguageModel(nn.Module):
  def __init__(self, vocabulary_size):
    super().__init__()
    self.token_embeddings_table = nn.Embedding(vocabulary_size, vocabulary_size)

  def forward(self, index, targets=None):
    logits = self.token_embeddings_table(index)

    if targets is None:
      loss = None
    else:
      B, T, C = logits.shape
      logits = logits.view(B*T, C)
      targets = targets.view(B*T)
      loss = func.cross_entropy(input=logits, target=targets)

    return logits, loss

  def generate(self, index, max_new_tokens):

    for bb in range(max_new_tokens):
      logits, loss = self.forward(index)

      logits = logits[:, -1, :]

      probs = func.softmax(logits, dim=-1)

      index_next = torch.multinomial(probs, num_samples=1)

      index = torch.cat((index, index_next), dim=1)
    return index

torch.manual_seed(23)
model = LanguageModel(vocabulary_size).to(device)
m = model.to(device)

context = torch.zeros((1, 1), dtype=torch.long, device=device)
generated_chars = decode(m.generate(context, max_new_tokens=500)[0].tolist())
print(generated_chars)

	sk-èœ⅝g·C7-₂¾ék£₂B
:!S•%/q5c⅛p﻿t*j C ?WL)•GU“mP”’d!TQ·⅛1™wH¾œykB 2s47Sm*g﻿A⅖wô-Wüb'ühpa(':£—₂#1
:0vü%35q1Vadzm/½⋀LBa—nytmTç⅖⅜•—PzmFXhk_!ZEE4·ocf4﻿—M1yJ─6’7⋀⅖⅖.L1CD¼G2,oLm™F⅝cd¼Z	ñX.6•-b86giX“x_Ti⋀RnT’⋀¼i;:*nbl6’PAFJ#•3﻿iP#”eJ─xECiènD¼A%i#x¼Gzxp#aK-¼C⅖)]]”EKjlj¼]U4*po82œGDi™r I,kôzl&hN⅛U™géè;.x=T⅛O(C?èñL¾&3₂⋀SN2﻿─•_£j¼'k%Q¼4MamV™x#!rA’z·1⅜¼£+s‘ñp‘ü	zm.3™Eh3mi3$[BI’çW8/[d-⋀A﻿*V!x:ç7™=8(fy2TRc“uH–35pq3”y$[rcT⋀HA	æLY_¾89·─₂4L─,côgYNG’⅜’z!ççH¾éBIP#57¼t/=ñ_J─¾‘G B]–pR/W/7₂ç!w;-⅖wY6U⋀bip/æ)G–4’﻿H41CWW⅖


In [ ]:
torch.manual_seed(23)
optimizer = torch.optim.AdamW(model.parameters(), lr=0.1)
# scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=5, gamma=0.5)

epochs = 10000

torch.manual_seed(23)
for epoch in range(epochs):
  # model.train()

  if epoch % 250 == 0:
    losses = estimate_loss()
    print(f"Loss: {losses['train']}")
  xb, yb = get_batch('train')

  logits, loss = model.forward(xb, yb)
  optimizer.zero_grad(set_to_none=True )
  loss.backward()
  optimizer.step()

  model.eval()

  if loss.item() < 1:
    print(loss.item())
    break
  # if epoch % 10000 == 0:
  #   print(epoch)
# with torch.inference_mode():
print(loss.item())

Выходные данные были обрезаны до нескольких последних строк (5000).
tensor([116232,  18912, 145828,  21923])
tensor([125818, 142204,  10760, 115331])
tensor([166191,  24791,  62582, 200930])
tensor([203789, 124780, 138258, 175085])
tensor([  3865, 192901, 113561,  88397])
tensor([ 54919, 207432, 117058,  27197])
tensor([ 49847,   9162, 165910,  59697])
tensor([140853, 131550,  69425, 187427])
tensor([197563, 195135, 194108, 160915])
tensor([140068,  12161,  92557, 176008])
tensor([ 51869, 215023, 111066,   5756])
tensor([ 39672, 153113,  24909, 128150])
tensor([140154, 192950,  61932, 196671])
tensor([110660,  70362, 203141, 110743])
tensor([ 45071,  49758, 150643, 189999])
tensor([105382, 150359, 135123, 195693])
tensor([ 65267, 119070, 110232, 131064])
tensor([ 70929, 159567,  46709, 186033])
tensor([ 37228, 148932, 176518, 153901])
tensor([126438, 189583,  60921, 204035])
tensor([216625, 175145,  97342,   9452])
tensor([138184, 137463,  77282, 201404])
tensor([ 16648, 132218, 132330

In [ ]:
context = torch.zeros((1, 1), dtype=torch.long, device=device)
generated_chars = decode(m.generate(context, max_new_tokens=500)[0].tolist())
print(generated_chars)

	rk  erghe tof ke-paperrettoco d anto win, rakmuend, acon wen fo stsoud rg Ade von nd pan's in g
onge stourade m, Liteny ms r t PzYous _,  te oreedesoumoutimiaisucoo t, Lourommyouco.”
Frangaralesir nens, as n leminow, 318Prangracein laby d frexplake se rald, ld l kepoke filere wexperede s he lwonofinglangnged Noley.’s  n. insamang steton angrs beres bjusthemioy Busthit?” 18 f sthiofaph.
hof aris che ” loun, ad hinde m b₂4Loncailinœ⅜’styo be creadmtlaster, ce n perd bun wo pl
Thenips tetthrken s t


In [ ]:
data[434+1:434+8+1]

tensor([81, 64,  2, 79, 74,  2, 62, 67])

In [ ]:
loss

tensor(2.2124, device='cuda:0', grad_fn=<NllLossBackward0>)

In [ ]:
# saved = model.state_dict()
# model.parameters = saved